In [3]:
import pandas as pd 
import numpy as np 
import sklearn.model_selection as sk
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import os
from argparse import ArgumentParser
import logging
from typing import NoReturn
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
import evaluation_scripts.eval_passengers_up as eval

In [131]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.9 MB/s eta 0:00:00a 0:00:010m


In [4]:
TRAIN_BUS_CSV_PATH = "data/train_bus_schedule.csv"
X_PASSENGER = "data/X_passengers_up.csv"
X_TRIP = "data/X_trip_duration.csv"
ENCODER = "windows-1255"
RANDOM_STATE = 42

In [5]:
train_bus = pd.read_csv(TRAIN_BUS_CSV_PATH, encoding=ENCODER)
x_passenger = pd.read_csv(X_PASSENGER, encoding=ENCODER)
x_trip_duration = pd.read_csv(X_TRIP, encoding=ENCODER)

In [6]:
train_bus = pd.read_csv(TRAIN_BUS_CSV_PATH, encoding=ENCODER)
x_passenger = pd.read_csv(X_PASSENGER, encoding=ENCODER)
sample_size = 0.05  # 5% of the data
xgboost_sample_size = 0.15
baseline = train_bus.sample(frac=sample_size, random_state=RANDOM_STATE)
remaining_data = train_bus.drop(baseline.index)

xgboost_sample = remaining_data.sample(frac=xgboost_sample_size, random_state=RANDOM_STATE)
xgboost_X = xgboost_sample[x_passenger.columns]
xgboost_y = xgboost_sample["passengers_up"]
remaining_data = remaining_data.drop(xgboost_sample.index)

In [7]:
xgboost_sample_test = xgboost_sample.copy()

In [8]:
xgboost_sample_test['station_name'] = xgboost_sample_test['station_name'].str.split(r'[ /\\]')
xgboost_X_exploded = xgboost_sample_test.explode('station_name').reset_index(drop=True)

In [9]:
word_count = xgboost_X_exploded.groupby("station_name")["passengers_up"].describe().sort_values(by = "50%")

In [10]:
xgboost_X_exploded

,trip_id,part,trip_id_unique_station,trip_id_unique,line_id,direction,alternative,cluster,station_index,station_id,station_name,arrival_time,door_closing_time,arrival_is_estimated,latitude,longitude,passengers_up,passengers_continue,mekadem_nipuach_luz,passengers_continue_menupach
0,310854,ב,310854b39,310854b,27087,1,#,חולון עירוני ומטרופוליני+תחרות חולון,39,36060,רוטשילד,14:43:00,14:43:00,False,32.02666,34.742650,0,7,2.250000,15.750000
1,310854,ב,310854b39,310854b,27087,1,#,חולון עירוני ומטרופוליני+תחרות חולון,39,36060,הרצל,14:43:00,14:43:00,False,32.02666,34.742650,0,7,2.250000,15.750000
2,314556,ג,314556c30,314556c,23076,2,0,אונו-אלעד,30,31674,רוטשילד,09:02:00,09:02:00,False,32.08004,34.884193,2,16,2.000000,32.000000
3,314556,ג,314556c30,314556c,23076,2,0,אונו-אלעד,30,31674,"קק""ל",09:02:00,09:02:00,False,32.08004,34.884193,2,16,2.000000,32.000000
4,210684,ב,210684b29,210684b,27004,2,0,תל אביב,29,21446,עירוני,20:22:16,NaN,True,32.09505,34.775550,0,4,0.545455,2.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112087,311414,א,311414a21,311414a,12172,2,0,"דרומי-ראשל""צ-חולון",21,36387,ההסתדרות,07:11:00,07:11:00,False,32.01649,34.782760,4,33,3.000000,99.000000
112088,311414,א,311414a21,311414a,12172,2,0,"דרומי-ראשל""צ-חולון",21,36387,מוהליבר,07:11:00,07:11:00,False,32.01649,34.782760,4,33,3.000000,99.000000
112089,313165,א,313165a40,313165a,15145,1,#,חולון עירוני ומטרופוליני+תחרות חולון,40,36388,תל,12:17:10,NaN,True,32.03237,34.767900,0,0,2.333333,0.000000
112090,313165,א,313165a40,313165a,15145,1,#,חולון עירוני ומטרופוליני+תחרות חולון,40,36388,גיבורים,12:17:10,NaN,True,32.03237,34.767900,0,0,2.333333,0.000000


In [11]:
top_pop_words = xgboost_X_exploded.groupby("station_name")["trip_id_unique_station"].nunique().sort_values().tail(100)

In [12]:
word_analysis = xgboost_X_exploded[xgboost_X_exploded["station_name"].isin(top_pop_words.index)].groupby("station_name")["passengers_up"].describe()

In [13]:
weight_words =list(word_analysis[word_analysis["75%"]> 2].index)

In [14]:
weight_words

['אצ"ל',
 'ביאליק',
 'ההגנה',
 'חזון',
 'לוינסקי',
 'סוקולוב',
 'עקיבא',
 'קניון',
 'רבי',
 'רכבת']

In [15]:
weight_words = weight_words + list(word_analysis[word_analysis["50%"]> 0].index)

In [16]:
weight_words

['אצ"ל',
 'ביאליק',
 'ההגנה',
 'חזון',
 'לוינסקי',
 'סוקולוב',
 'עקיבא',
 'קניון',
 'רבי',
 'רכבת',
 'אצ"ל',
 'ביאליק',
 'בלפור',
 'גשר',
 'ההגנה',
 'המלך',
 "ז'בוטינסקי",
 'חזון',
 'יוספטל',
 'כצנלסון',
 'לוינסקי',
 'סוקולוב',
 'עקיבא',
 'קניון',
 'רבי',
 'רוטשילד']

In [ ]:
for i, word in enumerate(weight_words, start=1):
    xgboost_X[f'x_{i}'] = xgboost_X['station_name'].str.contains(word, regex=False).astype(int)


In [ ]:
xgboost_X.columns

Index(['trip_id', 'part', 'trip_id_unique_station', 'trip_id_unique',
       'line_id', 'direction', 'alternative', 'cluster', 'station_index',
       'station_id', 'station_name', 'arrival_time', 'door_closing_time',
       'arrival_is_estimated', 'latitude', 'longitude', 'passengers_continue',
       'mekadem_nipuach_luz', 'passengers_continue_menupach', 'x_1', 'x_2',
       'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12',
       'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19', 'x_20', 'x_21',
       'x_22', 'x_23', 'x_24', 'x_25', 'x_26'],
      dtype='object')

In [ ]:
label_encoder = LabelEncoder()
xgboost_X['cluster'] = label_encoder.fit_transform(xgboost_X['cluster'])
xgboost_X['alternative'] = label_encoder.fit_transform(xgboost_X['alternative'])
xgboost_X['part'] = label_encoder.fit_transform(xgboost_X['part'])
xgboost_X['station_id'] = label_encoder.fit_transform(xgboost_X['station_id'])
xgboost_X['trip_id_unique'] = label_encoder.fit_transform(xgboost_X['trip_id_unique'])

In [ ]:
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:0000:0100:01


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, Point

In [ ]:
del xgboost_X["geometry"]
del xgboost_X["grid_id"]
xgboost_X['arrival_is_estimated'] = xgboost_X['arrival_is_estimated'].astype(int)

In [ ]:
xgboost_X['door_closing_time'] = pd.to_datetime(xgboost_X['door_closing_time'])
xgboost_X['arrival_time'] = pd.to_datetime(xgboost_X['arrival_time'])

# Create door delta columns
xgboost_X["door_close_delta"] = None
mask_notna = xgboost_X["door_closing_time"].notna()
xgboost_X.loc[mask_notna, 'door_close_delta'] = (xgboost_X.loc[mask_notna, 'door_closing_time'] - xgboost_X.loc[mask_notna, 'arrival_time']).dt.total_seconds()
door_delta_mean = xgboost_X["door_close_delta"].mean()
xgboost_X["door_close_delta"] = xgboost_X["door_close_delta"].fillna(door_delta_mean)

# Categorize arrival time
arrival_hours = xgboost_X['arrival_time'].dt.hour
percentiles = arrival_hours.describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
percentile_values = percentiles.loc[
    ['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%']
].values
labels = [f'{int(value)}' for value in percentile_values]
labels.insert(0, '0')
xgboost_X['arrival_time_label'] = pd.cut(arrival_hours,
                                    bins=[0] + list(percentile_values) + [24],
                                    labels=labels,
                                    include_lowest=True)

/var/folders/24/vfbkhj7j0t142rz214mskjkr0000gn/T/ipykernel_67249/1107517897.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xgboost_X["door_close_delta"] = xgboost_X["door_close_delta"].fillna(door_delta_mean)


In [ ]:
index_to_drop = xgboost_y[xgboost_y>30].index
xgboost_y = xgboost_y.drop(index_to_drop)
xgboost_X = xgboost_X.drop(index_to_drop)

In [152]:
xgboost_X["arrival_time_label"] = xgboost_X["arrival_time_label"].astype(int)

In [160]:
del xgboost_X["arrival_time"]
del xgboost_X["door_closing_time"]
del xgboost_X["station_name"]# 
del xgboost_X["trip_id_unique_station"]

In [161]:
def xg_boost(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series,
             y_test: pd.Series):
    results = []

    # Define parameter grids to iterate over
    parameters = [
        {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 100},
        {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200},
        {'max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 100}
    ]

    for params in parameters:
        # Create XGBoost model
        model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', **params)

        # Train the model
        model.fit(X_train, y_train)

        # Predict on test set
        y_pred = model.predict(X_test)

        # Calculate RMSE
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        results.append((params, rmse))

    # Return results for further analysis or selection
    return results

In [162]:
X_train, X_test, y_train, y_test  = sk.train_test_split(xgboost_X,xgboost_y,train_size=0.75,random_state=RANDOM_STATE)

In [163]:
result = xg_boost(X_train, X_test, y_train, y_test )

/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/envs/iml.env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6

In [164]:
result

[({'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
  1.5081256191087256),
 ({'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100},
  1.4807873755485912),
 ({'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 100},
  1.4843717914668078),
 ({'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 200},
  1.4730636334386578),
 ({'max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 100},
  1.4607775302881258)]

In [13]:
import pandas as pd
X_PASSENGER = "data/X_passengers_up.csv"
X_TRIP = "data/X_trip_duration.csv"

In [14]:
x_passenger = pd.read_csv(X_PASSENGER,encoding="windows-1255")
x_trip = pd.read_csv(X_TRIP,encoding="windows-1255")

In [15]:
x_columns = x_trip.columns

In [16]:
list(x_columns)

['trip_id',
 'part',
 'trip_id_unique_station',
 'trip_id_unique',
 'line_id',
 'direction',
 'alternative',
 'cluster',
 'station_index',
 'station_id',
 'station_name',
 'arrival_time',
 'door_closing_time',
 'arrival_is_estimated',
 'latitude',
 'longitude',
 'passengers_up',
 'passengers_continue',
 'mekadem_nipuach_luz',
 'passengers_continue_menupach']